In [17]:
import pandas as pd
import glob
import json
from gensim.models import KeyedVectors
from collections import Counter
from nltk.tokenize import regexp_tokenize

In [18]:
paths = glob.glob('./articles_data/1/*.json')
paths = paths[:1000]

In [19]:
article_data = []
for path in paths:
    with open(path) as f:
        article_data.append(json.load(f))

In [20]:
article_data[0].keys()

dict_keys(['organizations', 'uuid', 'thread', 'author', 'url', 'ord_in_thread', 'title', 'locations', 'entities', 'highlightText', 'language', 'persons', 'text', 'external_links', 'published', 'crawled', 'highlightTitle'])

In [21]:
len(article_data)

1000

In [22]:
sections = []
for i in range(1000):
    sections.append(article_data[i]['thread']['section_title'])
Counter(sections)

Counter({'Business Wire Technology: Mobile/Wireless News': 11,
         'industriesnews.net  - Breaking Technology News latest RSS headlines': 60,
         'Technology: newsR': 29,
         "News24, South Africa's premier news source, provides breaking news on national, world, Africa, sport, entertainment, technology &amp; more. ": 47,
         'Electronic Products Technology Center Articles': 2,
         'Technology - Bloomberg Business': 8,
         'Technology News | afr.com': 5,
         'Reuters: Technology': 30,
         'FreshNews.com - Technology News': 87,
         'Technology: Electronics Press Releases and News - PR.com': 1,
         'FCW: The business of federal technology': 2,
         'Medical Design Technology | The Medical Design Engineer’s Resource for Products &amp; Technologies': 8,
         'mirror - Technology': 4,
         'Newswise Feature Channel: Technology': 2,
         'Moneycontrol Technology News': 2,
         'Government Technology': 1,
         'Gizmag Em

In [23]:
article_data[0]

{'organizations': [],
 'uuid': 'ea5c49d229ded1a94679a075702e79d3a0a8848e',
 'thread': {'social': {'gplus': {'shares': 0},
   'pinterest': {'shares': 0},
   'vk': {'shares': 0},
   'linkedin': {'shares': 0},
   'facebook': {'likes': 0, 'shares': 0, 'comments': 0},
   'stumbledupon': {'shares': 0}},
  'site_full': 'www.businesswire.com',
  'main_image': '',
  'site_section': 'http://feed.businesswire.com/rss/home/?rss=G1QFDERJXkJeGVtQXg==',
  'section_title': 'Business Wire Technology: Mobile/Wireless News',
  'url': 'http://www.businesswire.com/news/home/20150921006305/en/MobileSmith-4.0-Leap-Enterprise-App-Development-REST',
  'country': 'US',
  'title': 'MobileSmith 4.0 a Leap Forward in Enterprise App Development with REST Services, OAuth',
  'performance_score': 0,
  'site': 'businesswire.com',
  'participants_count': 0,
  'title_full': 'MobileSmith 4.0 a Leap Forward in Enterprise App Development with REST Services, OAuth',
  'spam_score': 0.0,
  'site_type': 'news',
  'published':

In [36]:
data

[{'full_title': 'Business Wire Technology: Mobile/Wireless News MobileSmith 4.0 a Leap Forward in Enterprise App Development with REST Services, OAuth',
  'url': 'http://www.businesswire.com/news/home/20150921006305/en/MobileSmith-4.0-Leap-Enterprise-App-Development-REST'},
 {'full_title': 'industriesnews.net  - Breaking Technology News latest RSS headlines GOPs Rubio says Minecraft can prepare kids for future',
  'url': 'http://www.industriesnews.net/story?sid=s236424245&ampcid=400'},
 {'full_title': 'industriesnews.net  - Breaking Technology News latest RSS headlines Wipro hires another TCS executive',
  'url': 'http://www.industriesnews.net/story?sid=s236286643&ampcid=400'},
 {'full_title': 'Technology: newsR Donald Trump Is Raped and Murdered in Newest “South Park” Episode - newsR',
  'url': 'http://newsr.in/n/Technology/75589he9j/Donald-Trump-Is-Raped-and-Murdered-in-Newest.htm'},
 {'full_title': "News24, South Africa's premier news source, provides breaking news on national, worl

In [26]:
def process_text(text):
    text = text.replace("\n"," ").replace("\r"," ")
    
    punc_list = '!"#$%()*+,-./:;<=>?@^_{|}~'
    t = str.maketrans(dict.fromkeys(punc_list," "))
    text = text.translate(t)
    
    t = str.maketrans(dict.fromkeys("'`",""))
    text = text.translate(t)
    
    words = regexp_tokenize(text,pattern='\s+',gaps=True)
    
    return words

In [27]:
process_text("Technology News | afr.com Business backs Malcolm Turnbull's 'fresh' look at reform | afr.com")

['Technology',
 'News',
 'afr',
 'com',
 'Business',
 'backs',
 'Malcolm',
 'Turnbulls',
 'fresh',
 'look',
 'at',
 'reform',
 'afr',
 'com']

In [39]:
data = []
for i in range(len(article_data)):
    data.append({
        'full_title':article_data[i]['thread']['section_title']+' '+article_data[i]['thread']['title_full'],
        'url':article_data[i]['thread']['url'],
        'title_tokens':process_text(article_data[i]['thread']['section_title']+' '+article_data[i]['thread']['title_full'])
        }
    )

In [ ]:
model = KeyedVectors.load_word2vec_format("./GoogleNews-vectors-negative300.bin",binary=True,limit=10**5)